In [68]:
from transformers import pipeline
from tqdm import tqdm
import sys
import json
sys.path.append('../../')
import globals

fname = "BioASQ-task11bPhaseB-testset3.json"
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/{fname}'
submission_file = json.load(open(test_batch_doc))

In [69]:
chat_gpt_fn = f'{globals.PATH.home}/data/11b_testset/11B_batch3_gpt.json'
chat_gpt = json.load(open(chat_gpt_fn, "r"))
chatgpt_questions = dict((elem['id'],elem) for elem in chat_gpt['questions'])

In [53]:
qa_model = pipeline("question-answering")
question = "What is AUROC in context of predictive modeling?"
context = "The predictive accuracy of an individual biomarker on cognitive impairment was evaluated using area under the receiver operating characteristic curve (AUROC), and multivariate logistic regression was applied to evaluate predictive accuracy of biomarkers on cognitive impairment; 178 subjects (41 PD, 31 VaD and 106 normal controls) were included."
qa_model(question = question, context = context)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.914626955986023,
 'start': 110,
 'end': 149,
 'answer': 'receiver operating characteristic curve'}

In [54]:
fname = "BioASQ-task11bPhaseB-testset3.json"
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/{fname}'
submission_file = json.load(open(test_batch_doc))

for sample in submission_file['questions']:
    context = ""
    question = sample["body"]
    q_type = sample["type"]
    options = []
    
    for snippet in sample["snippets"]:
        context += snippet["text"]
        
    res = qa_model(question = question, context = context)
    options.append((res["score"],res["answer"]))
    
    shortans = chatgpt_questions[sample['id']]['ideal']
    if len(shortans) > 5:
        res = qa_model(question = question, context = shortans)
        options.append((res["score"],res["answer"]))
    
    longans = chatgpt_questions[sample['id']]['short']
    if len(longans) > 5:
        res = qa_model(question = question, context = longans)
        options.append((res["score"],res["answer"]))
    
    print(sample["body"])
    print(sample["id"])
    print(options)
    best_ideal_ans = max(options,key=lambda item:item[0])
    print(best_ideal_ans)
    print('__________________________________')
    sample["ideal_answer"] = best_ideal_ans[1]

Is erenumab effective for trigeminal neuralgia?
63f73f1b33942b094c000008
[(0.16242441534996033, 'Erenumab did not reduce pain intensity'), (0.08406951278448105, 'further research is needed to determine the long-term efficacy'), (0.25706052780151367, 'erenumab has been found to be effective')]
(0.25706052780151367, 'erenumab has been found to be effective')
__________________________________
What is the first indication for lurasidone?
643d41e757b1c7a315000037
[(0.8408240675926208, 'Latuda®'), (0.9389773607254028, 'Lurasidone'), (0.7588313221931458, 'Lurasidone')]
(0.9389773607254028, 'Lurasidone')
__________________________________
Can other vaccines be given with COVID-19 vaccine?
643c88a257b1c7a315000030
[(0.050089024007320404, 'randomly assigned (1:1)'), (0.08731797337532043, 'other vaccines can be given with the COVID-19 vaccine.'), (0.05235554650425911, 'other vaccines can be given with the COVID-19 vaccine')]
(0.08731797337532043, 'other vaccines can be given with the COVID-19 va

Which are the types of myotonic dystrophy?
644291fa57b1c7a31500005f
[(0.11594507098197937, 'DM type 2 (DM2)'), (0.02982873097062111, 'myotonic dystrophy type 2 (DM2)'), (0.892813503742218, 'type 1 (DM1) and type 2 (DM2)')]
(0.892813503742218, 'type 1 (DM1) and type 2 (DM2)')
__________________________________
What are the active components of Opdualag?
63f741a633942b094c000009
[(0.16390417516231537, 'nivolumab AND relatlimab'), (0.7335296273231506, 'an opioid agonist and an opioid antagonist'), (0.01545773446559906, 'a workflow engine, a data analytics engine, an AI engine')]
(0.7335296273231506, 'an opioid agonist and an opioid antagonist')
__________________________________
Where in the body would  Schlemm's canal be found
640f8641201352f04a00002c
[(0.002060363069176674, 'eyes treated with excisional LOTAIM'), (0.3556532561779022, 'the eye'), (0.4171258509159088, 'the eye')]
(0.4171258509159088, 'the eye')
__________________________________
Can reinfection occur after SARS-CoV-2 infe

What are the most common neuropsychiatric disorders associated with Duchenne muscular dystrophy?
641794e7690f196b51000036
[(0.018690751865506172, 'obsessive-compulsive disorder (OCD)'), (0.017811719328165054, 'anxiety, depression, attention-deficit/hyperactivity disorder'), (0.0599672868847847, 'anxiety, depression, attention deficit hyperactivity disorder')]
(0.0599672868847847, 'anxiety, depression, attention deficit hyperactivity disorder')
__________________________________
Which interleukins are affected by Dupilumab?
63ee5c78f36125a426000001
[(0.2897609770298004, 'IL)-4 and IL-13'), (0.9879094362258911, 'IL-4 and IL-13'), (0.9790201783180237, 'IL-4 and IL-13')]
(0.9879094362258911, 'IL-4 and IL-13')
__________________________________
What cells are used to regenerate cartilage?
64316f8e57b1c7a315000016
[(0.773239016532898, 'Mesenchymal stem cells'), (0.6978467106819153, 'Chondrocytes'), (0.9990328550338745, 'Chondrocytes')]
(0.9990328550338745, 'Chondrocytes')
___________________

How is amniotic fluid produced?
6431f6a857b1c7a315000018
[(0.09900641441345215, 'by the flow of fluid from the fetal lung and bladder'), (0.167649045586586, 'by the fetus, the placenta, and the amniotic membrane'), (0.7002118229866028, 'by the fetal kidneys and lungs')]
(0.7002118229866028, 'by the fetal kidneys and lungs')
__________________________________
Which cancer is the BCG vaccine used for?
6415c252690f196b51000011
[(0.4430887699127197, 'non-muscle invasive bladder cancer'), (0.5125412344932556, 'tuberculosis'), (0.8252717852592468, 'tuberculosis')]
(0.8252717852592468, 'tuberculosis')
__________________________________
What are the most common congenital muscular dystrophies?
641791b9690f196b51000030
[(0.020207572728395462, 'dystroglycanopathies'), (0.03770307078957558, 'merosin-deficient CMD'), (0.047638148069381714, 'Duchenne muscular dystrophy, Becker muscular dystrophy')]
(0.047638148069381714, 'Duchenne muscular dystrophy, Becker muscular dystrophy')
____________________

In [55]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer,aggregation_strategy='max') # pass device=0 if using gpu
sentiment_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [56]:
res = pipe("""The patient reported no recurrence of palpitations at follow-up 6 months after the ablation.""")
print(res)

[{'entity_group': 'Sign_symptom', 'score': 0.9999311, 'word': 'palpitations', 'start': 38, 'end': 50}, {'entity_group': 'Clinical_event', 'score': 0.99975544, 'word': 'follow', 'start': 54, 'end': 60}, {'entity_group': 'Date', 'score': 0.999867, 'word': '6 months after', 'start': 64, 'end': 78}]


In [57]:
for sample in submission_file['questions']:    
    q_type = sample["type"]
    query = sample["ideal_answer"]
    
    if q_type == 'yesno':
        yesno_exact = chatgpt_questions[sample['id']]['exacta'] 
        sent = sentiment_model(query)[0]['label']
        
        if sent == 'NEG': 
            sample["exact_answer"] = 'no'
                
        if sent == 'POS':
            sample["exact_answer"] = 'yes'
                
        if sent == 'NEU':
            sample["exact_answer"] = yesno_exact

In [59]:
for sample in submission_file['questions']:    
    q_type = sample["type"]
    query = sample["ideal_answer"]

    res = pipe(query)
    entities = [ ent['word'] for ent in res ]
    chatgpt_ents = chatgpt_questions[sample['id']]['exacta'] 
    
    if q_type == 'list':
        sample["exact_answer"] = [ [e] for e in entities ]
        print(sample['body'])
        print(query)
        print(entities)
        print(chatgpt_ents)
        
    if q_type == 'factoid':
        sample["exact_answer"] =  [ [e] for e in entities ]
        print(sample['body'])
        print(query)
        print(entities)
        print(chatgpt_ents)
    print()


What is the first indication for lurasidone?
Lurasidone
['lurasidone']
['Major Depressive Disorder']







How many injections of CLS-TA did the patients participating in the PEACHTREE trial receive?
three
['three']
['PEACHTREE trial', 'injections', 'CLS-TA', 'patients', 'confidence']

What are the most commonly used diagnostic tests for the diagnosis of Duchenne muscular dystrophy?
genetic testing, muscle biopsy, and electromyography (EMG)
['genetic testing', 'muscle', 'electromyography']
['Creatine Kinase Test', 'DNA Testing', 'Muscle Biopsy', 'Electromyography']


is prosopagnosia inherited or  acquired?
acquired
['acquired']
['Inherited', 'Acquired', 'Genetic', 'Hereditary', 'Congenital']

Where are promoters typically found in DNA?
directly upstream or at the 5' end of the transcription initiation site
['upstream', "the 5'end of the transcription initiation site"]
['Promoter', 'Transcription Start Site', 'Transcription Factor Binding Site', 'Core Promoter', 'Enhancer']

What was

SER-109 is developed for prevention of which disease?
Clostridioides difficile infection
['clostridioides', 'difficile', 'infection']
Answer: [1] Malaria, [2] Dengue, [3] Zika, [4] Chikungunya, [5] Yellow Fever



Where is the DMD gene located?
Xp21
['xp21']
1. Chromosome X 
2. Xq21.2-q21.31 
3. X-linked 
4. Duchenne Muscular Dystrophy 
5. Dystrophin



Which form of breast cancer has Keytruda been FDA approved for?
pembrolizumab
['pembrolizumab']
Answer: 
1. Triple-Negative Breast Cancer 
2. Metastatic Breast Cancer 
3. HER2-Positive Breast Cancer 
4. Advanced Breast Cancer 
5. Early-Stage Breast Cancer





In [60]:
name = f'{fname.replace(".json","")}_ensamble.json'
submission_file_name = f'{globals.PATH.home}/data/processed/{name}'

with open(submission_file_name, "w") as outfile:
    json.dump(submission_file, outfile, indent=2)

In [63]:
submission_file_name

'/opt/bioasq/col-un-bioasq11/data/processed/BioASQ-task11bPhaseB-testset3_ensamble.json'

# ChatGPT Submission

In [64]:
fname = "BioASQ-task11bPhaseB-testset3.json"
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/{fname}'
submission_file = json.load(open(test_batch_doc))

for sample in submission_file['questions']:
    context = ""
    question = sample["body"]
    q_type = sample["type"]
    options = []
    
    shortans = chatgpt_questions[sample['id']]['ideal']
    if len(shortans) > 5:
        res = qa_model(question = question, context = shortans)
        options.append((res["score"],res["answer"]))
    
    longans = chatgpt_questions[sample['id']]['short']
    if len(longans) > 5:
        res = qa_model(question = question, context = longans)
        options.append((res["score"],res["answer"]))
    
    if len(options) == 0:
        for snippet in sample["snippets"]:
            context += snippet["text"]      
        res = qa_model(question = question, context = context)
        options.append((res["score"],res["answer"]))
        
    best_ideal_ans = max(options,key=lambda item:item[0])
    sample["ideal_answer"] = best_ideal_ans[1]

In [65]:
for sample in submission_file['questions']:    
    q_type = sample["type"]
    query = sample["ideal_answer"]
    
    if q_type == 'yesno':
        yesno_exact = chatgpt_questions[sample['id']]['exacta'] 
        sample["exact_answer"] = yesno_exact
    
        q_type = sample["type"]
        
    query = sample["ideal_answer"]

    chatgpt_ents = chatgpt_questions[sample['id']]['exacta']
    
    if q_type == 'list':
        sample["exact_answer"] = [ [e] for e in chatgpt_ents ]
        
    if q_type == 'factoid':
        sample["exact_answer"] = [ [e] for e in chatgpt_ents ]

In [66]:
name = f'{fname.replace(".json","")}_chatgpt.json'
submission_file_name = f'{globals.PATH.home}/data/processed/{name}'

with open(submission_file_name, "w") as outfile:
    json.dump(submission_file, outfile, indent=2)

In [67]:
submission_file_name 

'/opt/bioasq/col-un-bioasq11/data/processed/BioASQ-task11bPhaseB-testset3_chatgpt.json'

# Without ChatGPT

In [44]:
fname = "BioASQ-task11bPhaseB-testset3.json"
test_batch_doc = f'{globals.PATH.home}/data/11b_testset/{fname}'
submission_file = json.load(open(test_batch_doc))

for sample in submission_file['questions']:
    context = ""
    question = sample["body"]
    q_type = sample["type"]
    options = []
    
    for snippet in sample["snippets"]:
        context += snippet["text"]     
        
    res = qa_model(question = question, context = context)
    options.append((res["score"],res["answer"]))

    best_ideal_ans = max(options,key=lambda item:item[0])
    sample["ideal_answer"] = best_ideal_ans[1]

In [45]:
for sample in submission_file['questions']:    
    q_type = sample["type"]
    query = sample["ideal_answer"]
    
    if q_type == 'yesno':
        sent = sentiment_model(query)[0]['label']
        
        if sent == 'NEG': 
            sample["exact_answer"] = 'no'
                
        if sent == 'POS':
            sample["exact_answer"] = 'yes'
                
        if sent == 'NEU':
            sample["exact_answer"] = 'yes'
    
    res = pipe(query)
    entities = [ ent['word'] for ent in res ]
    
    if q_type == 'list':
        sample["exact_answer"] = entities[0:5]
        
    if q_type == 'factoid':
        sample["exact_answer"] = entities[0:5]

In [46]:
name = f'{fname.replace(".json","")}_no_chatgpt.json'
submission_file_name = f'{globals.PATH.home}/data/processed/{name}'

with open(submission_file_name, "w") as outfile:
    json.dump(submission_file, outfile, indent=2)

In [48]:
submission_file_name

'/opt/bioasq/col-un-bioasq11/data/processed/BioASQ-task11bPhaseB-testset3_no_chatgpt.json'